# Muscle segmentation L3 volume

In [1]:
import torch 
from Unet3plus.unet3plus import UNet_3Plus_3D

model_3D  = UNet_3Plus_3D(n_classes=1, in_channels=1, feature_scale=4)
model_3D.load_state_dict(torch.load('models/best_unet3plusmodel.pth', weights_only=True, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
from torchsummary import summary

summary(model_3D, input_size=(1, 19, 256, 256))